Fix random seed

In [1]:
import random
import torch
import numpy

random.seed(3407)
torch.manual_seed(3407)
torch.cuda.manual_seed_all(3407)
numpy.random.seed(3407)

Build dataset and dataloader

In [2]:
import torchvision
torchvision.__version__

'0.16.2'

In [3]:
from data import get_dataloader, get_normparam
!python data.py

src in dataset: size= <built-in method size of Tensor object at 0x7ff3da679c60> mode= <built-in method mode of Tensor object at 0x7ff3da679c60>
tgt in dataset: size= <built-in method size of Image object at 0x7ff3da679d50> mode= <built-in method mode of Image object at 0x7ff3da679d50>
src shape in dataloader: torch.Size([4, 1, 256, 256])
tgt shape in dataloader: torch.Size([4, 1, 256, 256])


In [4]:
mean_train, std_train = get_normparam('/data/NEU_Seg-main', True)
mean_test, std_test = get_normparam('/data/NEU_Seg-main', False)
print('train:', mean_train, std_train)
print('test:', mean_test, std_test)

train: 0.4454502735091958 0.1425343463182748
test: 0.42473554009837755 0.14345889458209693


In [5]:
from u2net import U2NETP
model = U2NETP(1, 4).cuda()
from torchsummary import summary
summary(model, input_size=(1, 256, 256))
# from torch import rand
# src = rand((1, 1, 256, 256)).cuda()
# tgt = model(src)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]             640
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
          REBNCONV-4         [-1, 64, 256, 256]               0
            Conv2d-5         [-1, 16, 256, 256]           9,232
       BatchNorm2d-6         [-1, 16, 256, 256]              32
              ReLU-7         [-1, 16, 256, 256]               0
          REBNCONV-8         [-1, 16, 256, 256]               0
         MaxPool2d-9         [-1, 16, 128, 128]               0
           Conv2d-10         [-1, 16, 128, 128]           2,320
      BatchNorm2d-11         [-1, 16, 128, 128]              32
             ReLU-12         [-1, 16, 128, 128]               0
         REBNCONV-13         [-1, 16, 128, 128]               0
        MaxPool2d-14           [-1, 16,

Train

In [6]:
from engine import train_one_epoch, test_one_epoch, CELoss
import transform as T
from u2net import U2NETP
model = U2NETP(1, 4).cuda()
print('model params:', sum(p.numel() for p in model.parameters()))
root = '/data/NEU_Seg-main'
trans_train = T.Compose([
    T.RandomCrop((0.3, 1.), (0.5, 2.)),
    T.Normalize(mean_train, std_train),
    T.RandomRotate((-180, 180), p=1.),
    T.RandomFlip(),
    T.Resize((256, 256)),
    T.GaussianNoise(0.01),
])
trans_test = T.Compose([
    T.Normalize(mean_test, std_test),
    T.Resize((256, 256)),
])
train_dataloader = get_dataloader(root, istrain=True, batch_size=12, trans=trans_train)
test_dataloader = get_dataloader(root, istrain=False, batch_size=1, trans=trans_test)
critirion = CELoss(reduction='mean').cuda()
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-3, weight_decay=1e-4)

for epochIdx in range(30):
    print(epochIdx + 1)
    train_one_epoch(model, train_dataloader, critirion, optimizer)
    if epochIdx % 1 == 0:
        test_one_epoch(model, test_dataloader, critirion)


model params: 1140508
1


Train: avgloss= 0.433606523946421


Val: avgloss= 0.2930463346198112 avgious= [0.9044163583083882, 0.0, 0.5776205451603534, 0.3136621265712651] miou= 0.29709422391053947
2


Train: avgloss= 0.304277792869025


Val: avgloss= 0.259687062620651 avgious= [0.9201488421101062, 0.03629287443284004, 0.6537769055211334, 0.367598887132172] miou= 0.3525562223620485
3


Train: avgloss= 0.2723679526505486


Val: avgloss= 0.2543224661976897 avgious= [0.9238764745714739, 0.014359051916080626, 0.6743468898696227, 0.4677588908067105] miou= 0.3854882775308046
4


Train: avgloss= 0.26618422891031823


Val: avgloss= 0.22822102540771344 avgious= [0.9302835784302652, 0.18131753978034362, 0.693066258703503, 0.536349704468542] miou= 0.47024450098412957
5


Train:  84%|████████▍ | 254/302 [01:30<00:18,  2.64it/s]